In [1]:
import gradio as gr
from PIL import Image
from dotenv import load_dotenv
import random
import cv2
import numpy as np
import google.generativeai as genai

load_dotenv()

genai.configure()

example_images = {
    "플랭크": "plank.jpg",
    "팔굽혀펴기": "pushup.jpg"
}

youtube_videos = {
    "플랭크": [
        "https://www.youtube.com/embed/1T6P56q1FtY",
        "https://www.youtube.com/embed/Zq8nRY9P_cM",
        "https://www.youtube.com/embed/iNdDN4GKtHk",
        "https://www.youtube.com/embed/v54Jtmi2BwU",
        "https://www.youtube.com/embed/N7Izxcyu2XI",
        "https://www.youtube.com/embed/1-KnmYQMsE8"
    ],
    "팔굽혀펴기": [
        "https://www.youtube.com/embed/IODxDxX7oi4",
        "https://www.youtube.com/embed/0pkjOk0EiAk",
        "https://www.youtube.com/embed/blzrPdzxzHQ",
        "https://www.youtube.com/embed/fh4y5dGZX9c",
        "https://www.youtube.com/embed/EHPMNaWNdPQ",
        "https://www.youtube.com/embed/rR4p13vgDFA"
    ]
}


# 제미나이 프롬프트
def analyze_pose_with_genai(image):
    vmodel = genai.GenerativeModel('gemini-pro-vision')
    response = vmodel.generate_content([
        "운동자세에서 잘못된 부분을 지적해 주세요. 번호를 매겨주세요.",
        image
    ])
    return response.text

# 유사도 비교
def process(exercise, image1, image2):
    # 미리 작성한 유튜브 영상 랜덤 추출
    youtube_url = random.choice(youtube_videos.get(exercise, []))
    youtube_html = f'<iframe width="560" height="315" src="{youtube_url}" frameborder="0" allowfullscreen></iframe>'

    # 이미지 -> 배열 변환
    uploaded_image = np.array(image1)
    # 회색조 적용
    if uploaded_image.ndim == 3 and uploaded_image.shape[2] == 3:
        uploaded_image = cv2.cvtColor(uploaded_image, cv2.COLOR_RGB2GRAY)

    # 예시 이미지 로드
    example_image_path = example_images.get(exercise)
    example_image = cv2.imread(example_image_path, cv2.IMREAD_GRAYSCALE)
    example_image = cv2.resize(example_image, (500, 500))

    # 히스토그램 값 계산 및 정규화
    example_hist = cv2.calcHist([example_image], [0], None, [256], [0, 256])
    uploaded_hist = cv2.calcHist([uploaded_image], [0], None, [256], [0, 256])
    example_hist = cv2.normalize(example_hist, example_hist).flatten()
    uploaded_hist = cv2.normalize(uploaded_hist, uploaded_hist).flatten()

    # 히스토그램 값 비교
    score = cv2.compareHist(example_hist, uploaded_hist, cv2.HISTCMP_CORREL)

    # 제미나이 피드백 생성
    pose_feedback = analyze_pose_with_genai(image1)
    
    # 점수에 따라 코멘트 제공
    if score < 0.7:
        message = "자세가 조금 불안정해요. 영상을 보고 연습해보세요!"
    else:
        message = "자세가 아주 좋아요! 더 완벽하게 할 수 있게 영상을 보고 연습해보세요!"

    # 유사도 계산
    result_message = f"유사도 점수: {score:.2f} - {message}" + f"자세 피드백: {pose_feedback}"

    return youtube_html, result_message

def show_example_image(exercise):
    example_image_path = example_images.get(exercise, None)
    if exercise == "팔굽혀펴기":
        return gr.update(visible=True), gr.update(visible=True), example_image_path, "왼쪽 이미지를 보고 올바른 자세로 사진을 찍어주세요."
    elif exercise == "플랭크":
        return gr.update(visible=True), gr.update(visible=False), example_image_path, "왼쪽 이미지를 보고 올바른 자세로 사진을 찍어주세요."
    else:
        return gr.update(visible=False), gr.update(visible=False), None, ""


with gr.Blocks() as blocks:
    with gr.Row():
        exercise_input = gr.Radio(label="어떤 운동인가요?", choices=["플랭크", "팔굽혀펴기"])
        
    with gr.Row():
        example_image_output = gr.Image(type="pil")
        example_description = gr.HTML()
    
    with gr.Row():
        image_input1 = gr.Image(label="이미지를 업로드해주세요.", visible=False, type="pil")
        image_input2 = gr.Image(label="이미지를 업로드해주세요.", visible=False, type="pil")
    
    with gr.Row():
        submit_button = gr.Button("Submit")
        
    with gr.Row():
        youtube_output = gr.HTML(label="유튜브 동영상")
        similarity_score = gr.HTML(label="유사도 점수")

    exercise_input.change(
        fn=show_example_image,
        inputs=exercise_input,
        outputs=[image_input1, image_input2, example_image_output, example_description]
    )

    submit_button.click(
        fn=process,
        inputs=[exercise_input, image_input1, image_input2],
        outputs=[youtube_output, similarity_score]
    )

blocks.launch()


c:\Users\JangSeokHwan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
